In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/data/change_df.csv',index_col=0)
df = df[['user_id','item_id','votes','timestamp','rating','compound_score']]
df.head()

,user_id,item_id,votes,timestamp,rating,compound_score
0,B00191WVF6,A0955928C2RRWOWZN7UC,0.0,2017-02-17,4.0,0.0000
1,B005WY3TMA,A0955928C2RRWOWZN7UC,0.0,2015-06-14,4.0,0.4404
2,B0090XWU8S,A0955928C2RRWOWZN7UC,0.0,2017-04-22,4.0,0.0000
3,B00FXYTLIK,A0955928C2RRWOWZN7UC,0.0,2015-07-10,4.0,0.7264
4,B00HMZG3YS,A0955928C2RRWOWZN7UC,0.0,2015-07-10,4.0,0.6369


In [4]:
df['rating'] = df['rating'].astype(int)
df['votes'] = df['votes'].astype(int)

In [5]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year']= df['timestamp'].dt.year

In [6]:
def scorechange(n):
  if n<=0.1 :
    r = 0
  else : 
    r = 1
  return r

In [7]:
def votechange(n):
  if n==0:
    r = 'not voted'
  else :
    r = 'voted'
  return r

In [8]:
df['votes'] = df['votes'].apply(votechange)

In [9]:
#df.rename(columns = {'compound_score' : 'sentiment'}, inplace = True)
df['sentiment'] = df['compound_score'].apply(scorechange)

In [10]:
df

,user_id,item_id,votes,timestamp,rating,compound_score,year,sentiment
0,B00191WVF6,A0955928C2RRWOWZN7UC,not voted,2017-02-17,4,0.0000,2017,0
1,B005WY3TMA,A0955928C2RRWOWZN7UC,not voted,2015-06-14,4,0.4404,2015,1
2,B0090XWU8S,A0955928C2RRWOWZN7UC,not voted,2017-04-22,4,0.0000,2017,0
3,B00FXYTLIK,A0955928C2RRWOWZN7UC,not voted,2015-07-10,4,0.7264,2015,1
4,B00HMZG3YS,A0955928C2RRWOWZN7UC,not voted,2015-07-10,4,0.6369,2015,1
...,...,...,...,...,...,...,...,...
99737,B009AYLDSU,AZZYW4YOE1B6E,voted,2013-12-10,5,0.9798,2013,1
99738,B00E055H5O,AZZYW4YOE1B6E,not voted,2015-05-25,4,0.8402,2015,1
99739,B00E8HGWIK,AZZYW4YOE1B6E,not voted,2013-12-01,5,0.9940,2013,1
99740,B00M58CMTM,AZZYW4YOE1B6E,voted,2014-10-08,5,0.9856,2014,1


In [49]:
from sklearn.utils import shuffle
df = shuffle(df, random_state=52)
cutoff = int(len(df)*0.7)
train_df = df[:cutoff]
test_df = df[cutoff:]

In [50]:
train_df.head()

,user_id,item_id,votes,timestamp,rating,compound_score,year,sentiment
17984,B005LFT3QG,A1PV5T5S4D5NPI,not voted,2015-09-30,5,0.5413,2015,1
74332,B015QUIGEU,A3TJF7TNFIWQDO,not voted,2017-03-27,5,0.8655,2017,1
97175,B005FHZ2SI,AVV5AKK829T3K,not voted,2017-04-20,3,-0.6513,2017,0
18547,B00005ATMI,A1QQ4U4RTI0A1M,not voted,2015-07-23,5,0.0000,2015,0
70325,B008BGXHRS,A3O4MIGSZV0LP7,voted,2015-07-07,5,0.9199,2015,1


In [51]:
# train set에 있는 아이템별 감성점수 평균
mean_sentiment_score = train_df.groupby(['item_id']).mean()['compound_score'].to_frame()
mean_sentiment_score

,compound_score
item_id,
A0955928C2RRWOWZN7UC,0.376588
A100UD67AHFODS,0.852733
A100WO06OQR8BQ,0.474365
A1013Q9SD2KIE1,-0.009029
A1018D502E27E6,0.141940
...,...
AZX2RDN9YXZAE,0.060983
AZXFS8GCTSQ5R,0.930767
AZXJJM4F1SY3,0.552800


In [52]:
mean_sentiment_score.rename(columns = {'compound_score' : 'sentiment'}, inplace = True)

In [53]:
mean_sentiment_score['sentiment'] = mean_sentiment_score['sentiment'].apply(scorechange)
mean_sentiment_score

,sentiment
item_id,
A0955928C2RRWOWZN7UC,1
A100UD67AHFODS,1
A100WO06OQR8BQ,1
A1013Q9SD2KIE1,0
A1018D502E27E6,1
...,...
AZX2RDN9YXZAE,0
AZXFS8GCTSQ5R,1
AZXJJM4F1SY3,1


In [54]:
# train, test 셋에서 compound 삭제 (사용하지 않을 변수)
train_df = train_df.drop('compound_score',axis=1)
test_df = test_df.drop('compound_score',axis=1)

In [55]:
# test 셋에서 감성점수(sentiment) 삭제
test_df = test_df.drop('sentiment',axis=1)

In [56]:
len(test_df)

29923

In [57]:
# test 셋에 train set에 있는 감성점수 붙이기(치팅방지)
test_df = pd.merge(mean_sentiment_score,test_df,how='inner',on='item_id')
test_df.head()

,item_id,sentiment,user_id,votes,timestamp,rating,year
0,A0955928C2RRWOWZN7UC,1,B00HMZG3YS,not voted,2015-07-10,4,2015
1,A0955928C2RRWOWZN7UC,1,B00191WVF6,not voted,2017-02-17,4,2017
2,A0955928C2RRWOWZN7UC,1,B010OYASRG,not voted,2017-03-17,4,2017
3,A0955928C2RRWOWZN7UC,1,B00NIYJL64,not voted,2017-02-17,4,2017
4,A100UD67AHFODS,1,B0053T4PHC,voted,2013-01-05,5,2013


In [58]:
len(test_df)

29923

In [59]:
test_df.sentiment.value_counts()

1    26691
0     3232
Name: sentiment, dtype: int64

In [60]:
# User encoding
user_dict = {}
for i in set(df['user_id']):
    user_dict[i] = len(user_dict)
n_user = len(user_dict)
# Item encoding
item_dict = {}
start_point = n_user
for i in set(df['item_id']):
    item_dict[i] = start_point + len(item_dict)
n_item = len(item_dict)
start_point += n_item


# vote encoding
vote_dict = {}
for i in set(df['votes']):
    vote_dict[i] = start_point + len(vote_dict)
n_vote = len(vote_dict)
start_point += n_vote

#year encoding
year_dict = {}
for i in set(df['year']):
    year_dict[i] = start_point + len(year_dict)
n_year = len(year_dict)
start_point += n_year

# sentiment encoding
sentiment_dict = {}
for i in set(df['sentiment']):
    sentiment_dict[i] = start_point + len(sentiment_dict)
n_sentiment = len(sentiment_dict)
start_point += n_sentiment

In [61]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [62]:
class FM():
    def __init__(self, N, K, train_data, test_data, train_y, test_y, alpha, beta, iterations=100, tolerance=0.005, l2_reg=True, verbose=True):
        self.K = K          # Number of latent factors
        self.N = N          # Number of x (variables)
        # self.n_cases = len(data)            # N of observations
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.l2_reg = l2_reg
        self.tolerance = tolerance
        self.verbose = verbose
        # w 초기화
        self.w = np.random.normal(scale=1./self.N, size=(self.N))
        # v 초기화
        self.v = np.random.normal(scale=1./self.K, size=(self.N, self.K))
        # Train/Test 분리
        self.train_x = train_data
        self.test_x = test_data
        self.train_y = train_y
        self.test_y = test_y
   
    def test(self):                                     # Training 하면서 RMSE 계산 
        # SGD를 iterations 숫자만큼 수행
        best_RMSE = 10000
        best_iteration = 0
        training_process = []
        best_pred = []
        for i in range(self.iterations):
            rmse1 = self.sgd(self.train_x, self.train_y)        # SGD & Train RMSE 계산
            y_pred, rmse2 = self.test_rmse(self.test_x, self.test_y)    # Test RMSE 계산    
            training_process.append((i, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.6f ; Test RMSE = %.6f" % (i+1, rmse1, rmse2))
            if best_RMSE > rmse2:                       # New best record
                best_RMSE = rmse2
                best_iteration = i
                best_pred = y_pred
            elif (rmse2 - best_RMSE) > self.tolerance:  # RMSE is increasing over tolerance
                break
        print(f'-----fm_test : {best_iteration}, {best_RMSE}')
        return training_process, best_pred, best_RMSE
        
    # w, v 업데이트를 위한 Stochastic gradient descent 
    def sgd(self, x_data, y_data):
        y_pred = []
        for data, y in zip(x_data, y_data):
            x_idx = data[0]
            x_0 = np.array(data[1])     # xi axis=0 [1, 2, 3]
            x_1 = x_0.reshape(-1, 1)    # xi axis=1 [[1], [2], [3]]
    
            # biases
            bias_score = np.sum(self.w[x_idx] * x_0)
            
            # score 계산
            vx = self.v[x_idx] * (x_1)          # v matrix * x
            sum_vx = np.sum(vx, axis=0)         # sigma(vx)
            sum_vx_2 = np.sum(vx * vx, axis=0)  # ( v matrix * x )의 제곱
            latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2)

            # 예측값 계산
            y_hat = bias_score + latent_score
            y_pred.append(y_hat)
            error = y - y_hat
            # w, v 업데이트
            if self.l2_reg:     # regularization이 있는 경우
                self.w[x_idx] += error * self.alpha * (x_0 - self.beta * self.w[x_idx])
                self.v[x_idx] += error * self.alpha * ((x_1) * sum(vx) - (vx * x_1) - self.beta * self.v[x_idx])
            else:               # regularization이 없는 경우
                self.w[x_idx] += error * self.alpha * x_0
                self.v[x_idx] += error * self.alpha * ((x_1) * sum(vx) - (vx * x_1))
        return RMSE(y_data, y_pred)
            
    def test_rmse(self, x_data, y_data):
        y_pred = []
        for data , y in zip(x_data, y_data):
            y_hat = self.predict(data[0], data[1])
            y_pred.append(y_hat)
        return np.array(y_pred), RMSE(y_data, y_pred)

    def predict(self, idx, x):
        # idx = self.user_id_index[idx]
        # x = self.item_id_index[x]
        x_0 = np.array(x)
        x_1 = x_0.reshape(-1, 1)

        # biases
        bias_score = np.sum(self.w[idx] * x_0)

        # score 계산
        vx = self.v[idx] * (x_1)
        sum_vx = np.sum(vx, axis=0)
        sum_vx_2 = np.sum(vx * vx, axis=0)
        latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2)

        # 예측값 계산
        y_hat = bias_score + latent_score
        return y_hat
    

In [63]:
# class NEW_MF():
#     # Initializing the object
#     def __init__(self, ratings, K, alpha, beta, iterations, tolerance=0.005, verbose=True):
#         self.R = np.array(ratings)
#         # user_id, movie_id를 R의 index와 매칭하기 위한 dictionary 생성
#         item_id_index = []
#         index_item_id = []
#         for i, one_id in enumerate(ratings):
#             item_id_index.append([one_id, i])
#             index_item_id.append([i, one_id])
#         self.item_id_index = dict(item_id_index)
#         self.index_item_id = dict(index_item_id)        
#         user_id_index = []
#         index_user_id = []
#         for i, one_id in enumerate(ratings.T):
#             user_id_index.append([one_id, i])
#             index_user_id.append([i, one_id])
#         self.user_id_index = dict(user_id_index)
#         self.index_user_id = dict(index_user_id)
#         # 다른 변수 초기화
#         self.num_users, self.num_items = np.shape(self.R)
#         self.K = K
#         self.alpha = alpha
#         self.beta = beta
#         self.iterations = iterations
#         self.tolerance = tolerance
#         self.verbose = verbose

#     # 테스트 셋을 선정하는 메소드 
#     def set_test(self, ratings_test):                           # Setting test set
#         test_set = []
#         for i in range(len(ratings_test)):                      # Selected ratings
#             x = self.user_id_index[ratings_test.iloc[i,0]]      # Getting R indice for the given user_id and movie_id
#             y = self.item_id_index[ratings_test.iloc[i,1]]
#             z = ratings_test.iloc[i,2]
#             test_set.append([x, y, z])
#             self.R[x, y] = 0                    # Setting test set ratings to 0
#         self.test_set = test_set
#         return test_set                         # Return test set

#     def test(self):                             # Training 하면서 test set의 정확도를 계산하는 메소드 
#         # Initializing user-feature and movie-feature matrix
#         self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
#         self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

#         # Initializing the bias terms
#         self.b_u = np.zeros(self.num_users)
#         self.b_d = np.zeros(self.num_items)
#         self.b = np.mean(self.R[self.R.nonzero()])

#         # List of training samples
#         rows, columns = self.R.nonzero()
#         self.samples = [(i,j, self.R[i,j]) for i, j in zip(rows, columns)]

#         # Stochastic gradient descent for given number of iterations
#         best_RMSE = 10000
#         best_iteration = 0
#         training_process = []
#         for i in range(self.iterations):
#             np.random.shuffle(self.samples)
#             self.sgd()
#             rmse1 = self.rmse()
#             rmse2 = self.test_rmse()
#             training_process.append((i, rmse1, rmse2))
#             if self.verbose:
#                 if (i+1) % 10 == 0:
#                     print("Iteration: %d ; Train RMSE = %.6f ; Test RMSE = %.6f" % (i+1, rmse1, rmse2))
#             if best_RMSE > rmse2:                      # New best record
#                 best_RMSE = rmse2
#                 best_iteration = i
#             elif (rmse2 - best_RMSE) > self.tolerance: # RMSE is increasing over tolerance
#                 break
#         print(f'-----mf_test :  {best_iteration}, {best_RMSE}')
#         return training_process

#     # Stochastic gradient descent to get optimized P and Q matrix
#     def sgd(self):
#         for i, j, r in self.samples:
#             prediction = self.get_prediction(i, j)
#             e = (r - prediction)

#             self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
#             self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

#             self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])
#             self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])

#     # Computing mean squared error
#     def rmse(self):
#         xs, ys = self.R.nonzero()
#         self.predictions = []
#         self.errors = []
#         for x, y in zip(xs, ys):
#             prediction = self.get_prediction(x, y)
#             self.predictions.append(prediction)
#             self.errors.append(self.R[x, y] - prediction)
#         self.predictions = np.array(self.predictions)
#         self.errors = np.array(self.errors)
#         return np.sqrt(np.mean(self.errors**2))

#     # Test RMSE 계산하는 method 
#     def test_rmse(self):
#         error = 0
#         for one_set in self.test_set:
#             predicted = self.get_prediction(one_set[0], one_set[1])
#             error += pow(one_set[2] - predicted, 2)
#         return np.sqrt(error/len(self.test_set))

#     # Ratings for user i and moive j
#     def get_prediction(self, i, j):
#         prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
#         return prediction

#     # Ratings for user_id and moive_id
#     def get_one_prediction(self, user_id, movie_id):
#         return self.get_prediction(self.user_id_index[user_id], self.item_id_index[movie_id])
    

In [64]:
# Total number of x
num_x = start_point             
num_x

10097

In [65]:
def generate_x(x) :
    # Generate X data
    data = []
    y = []
    w0 = np.mean(x['rating'])
    for i in range(len(x)):
        case = x.iloc[i]
        x_index = []
        x_value = []
        x_index.append(user_dict[case['user_id']])     # User id encoding
        x_value.append(1.)
        x_index.append(item_dict[case['item_id']])    # item id encoding
        x_value.append(1.)
        x_index.append(vote_dict[case['votes']])   # vote encoding
        x_value.append(1.)
        x_index.append(year_dict[case['year']])       # year encoding
        x_value.append(1.)
        x_index.append(sentiment_dict[case['sentiment']]) #sentiment encoding
        x_value.append(1.)

        data.append([x_index, x_value])
        y.append(case['rating'] - w0)
        if (i % 10000) == 0:
            print('Encoding ', i, ' cases...')
    return data, y

In [66]:
train_data, train_y = generate_x(train_df) 
test_data, test_y = generate_x(test_df) 

Encoding  0  cases...
Encoding  10000  cases...
Encoding  20000  cases...
Encoding  30000  cases...
Encoding  40000  cases...
Encoding  50000  cases...
Encoding  60000  cases...
Encoding  0  cases...
Encoding  10000  cases...
Encoding  20000  cases...


### 12

In [29]:
fm1 = FM(num_x, 260, train_data, test_data, train_y, test_y, alpha=0.0001, beta=0.007, iterations=900, tolerance=0.0005, l2_reg=True, verbose=True)
training_process, result0, rmse = fm1.test()  
print('FM RMSE:' , rmse)

Iteration: 10 ; Train RMSE = 0.953739 ; Test RMSE = 1.003446
Iteration: 20 ; Train RMSE = 0.938912 ; Test RMSE = 0.997878
Iteration: 30 ; Train RMSE = 0.929690 ; Test RMSE = 0.993683
Iteration: 40 ; Train RMSE = 0.918208 ; Test RMSE = 0.988318
Iteration: 50 ; Train RMSE = 0.902492 ; Test RMSE = 0.981190
Iteration: 60 ; Train RMSE = 0.881940 ; Test RMSE = 0.972561
Iteration: 70 ; Train RMSE = 0.858144 ; Test RMSE = 0.963849
Iteration: 80 ; Train RMSE = 0.834151 ; Test RMSE = 0.956861
Iteration: 90 ; Train RMSE = 0.812780 ; Test RMSE = 0.952825
Iteration: 100 ; Train RMSE = 0.795241 ; Test RMSE = 0.951803
-----fm_test : 98, 0.9517904732250666
FM RMSE: 0.9517904732250666


### 32

In [48]:
fm1 = FM(num_x, 260, train_data, test_data, train_y, test_y, alpha=0.0001, beta=0.007, iterations=900, tolerance=0.0005, l2_reg=True, verbose=True)
training_process, result0, rmse = fm1.test()  
print('FM RMSE:' , rmse)

Iteration: 10 ; Train RMSE = 0.964917 ; Test RMSE = 0.980025
Iteration: 20 ; Train RMSE = 0.950325 ; Test RMSE = 0.974361
Iteration: 30 ; Train RMSE = 0.940928 ; Test RMSE = 0.970145
Iteration: 40 ; Train RMSE = 0.929012 ; Test RMSE = 0.964393
Iteration: 50 ; Train RMSE = 0.912513 ; Test RMSE = 0.956447
Iteration: 60 ; Train RMSE = 0.890992 ; Test RMSE = 0.946654
Iteration: 70 ; Train RMSE = 0.866305 ; Test RMSE = 0.936847
Iteration: 80 ; Train RMSE = 0.841702 ; Test RMSE = 0.929289
Iteration: 90 ; Train RMSE = 0.820065 ; Test RMSE = 0.925279
Iteration: 100 ; Train RMSE = 0.802497 ; Test RMSE = 0.924593
-----fm_test : 96, 0.9245116220339962
FM RMSE: 0.9245116220339962


### 52

In [ ]:
fm1 = FM(num_x, 260, train_data, test_data, train_y, test_y, alpha=0.0001, beta=0.007, iterations=900, tolerance=0.0005, l2_reg=True, verbose=True)
training_process, result0, rmse = fm1.test()  
print('FM RMSE:' , rmse)

Iteration: 10 ; Train RMSE = 0.963038 ; Test RMSE = 0.982548


In [ ]:
fm1 = FM(num_x, 250, train_data, test_data, train_y, test_y, alpha=0.0002, beta=0.007, iterations=900, tolerance=0.0005, l2_reg=True, verbose=True)
training_process, result0, rmse = fm1.test()  
print('FM RMSE:' , rmse)

In [ ]:
# ratings = df.pivot(index = 'user_id', columns ='item_id', values = 'rating').fillna(0)
# mf = NEW_MF(ratings, K=16, alpha=0.0013, beta=0.28, iterations=150, tolerance=0.0001, verbose=True)
# test_x = test_df[['user_id','item_id','rating','votes','sentiment','year']]
# test_set = mf.set_test(test_x)
# result = mf.test()

Iteration: 10 ; Train RMSE = 0.942195 ; Test RMSE = 0.953470
Iteration: 20 ; Train RMSE = 0.909318 ; Test RMSE = 0.937739
Iteration: 30 ; Train RMSE = 0.886906 ; Test RMSE = 0.928492
Iteration: 40 ; Train RMSE = 0.870852 ; Test RMSE = 0.922838
Iteration: 50 ; Train RMSE = 0.858981 ; Test RMSE = 0.919357
Iteration: 60 ; Train RMSE = 0.850000 ; Test RMSE = 0.917263
Iteration: 70 ; Train RMSE = 0.843073 ; Test RMSE = 0.916060
Iteration: 80 ; Train RMSE = 0.837630 ; Test RMSE = 0.915418
Iteration: 90 ; Train RMSE = 0.833295 ; Test RMSE = 0.915161
Iteration: 100 ; Train RMSE = 0.829794 ; Test RMSE = 0.915152
-----mf_test :  94, 0.9151295244864293


In [ ]:
# def recommender1(recomm_list, mf):
#     recommendations = []
#     for i in range(len(recomm_list)):
#         recommendations.append(mf.get_one_prediction(recomm_list[i,0], recomm_list[i,1]))
#     return np.array(recommendations)

# # RMSE 계산을 위한 함수
# def RMSE2(y_true, y_pred):
#     return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# recomm_list = np.array(test_x.loc[:, ['user_id', 'item_id']])        # Data for testing context-DL model
# result1 = recommender1(recomm_list, mf)

In [ ]:
# # 가중치 별 테스트
# for i in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
#     weight = [i, 1-i]
#     predictions = []
#     w0 = np.mean(df['rating'])
#     for i, number in enumerate(result0):
#         predictions.append(result0[i] * weight[0] + (result1[i]-w0) * weight[1])
#     print("Weights - %.2f : %.2f ; RMSE = %.7f" % (weight[0], weight[1], RMSE2(test_y, predictions)))

In [ ]:
# # 가중치 별 테스트
# for i in [0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89]:
#     weight = [i, 1-i]
#     predictions = []
#     w0 = np.mean(df['rating'])
#     for i, number in enumerate(result0):
#         predictions.append(result0[i] * weight[0] + (result1[i]-w0) * weight[1])
#     print("Weights - %.2f : %.2f ; RMSE = %.7f" % (weight[0], weight[1], RMSE2(test_y, predictions)))